# Algoritmo de transposición de conjunto de datos generalizado

En este notebook se ha desarrollado un algoritmo que permite hacer transformaciones a datasets de *n* dimensiones. Con el siguiente ejemplo se va entender cómo funciona y cuáles són las motivaciones para ser desarrollado.

## Ejemplo

In [344]:
import numpy as np

# Se genera una conjunto de datos inicial

src = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])

# Se genera el conjunto de datos que queremos obtener si las subdimensiones fueran (2, 2)

dest = np.array([[1, 2, 5, 6], [3, 4, 7, 8], [9, 10, 13, 14], [11, 12, 15, 16]])

# Se imprime los resultados

print("\nLos datos originales son:\n")
print(src)

print("\nLos datos modificados son:\n")
print(dest)


Los datos originales son:

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]

Los datos modificados son:

[[ 1  2  5  6]
 [ 3  4  7  8]
 [ 9 10 13 14]
 [11 12 15 16]]


Como se observa, la idea de la trasformación es colocar próximos (en sentido lineal) los datos que están próximos teniendo en cuenta las dimensiones.

Por ejemplo, si consideramos las matrices como vectores, el número 2 tiene como elementos más próximos al 1 y al 3, pero si la consideramos como matriz tiene también al 5, 6, 7. sin embargo, si consideramos la matriz como vector estos elementos están alejados del 2.

Después de la transformación, vemos que el 5 y el 6, que antes estaban lejos, ahora están más próximos. Cuanto más grande son los tamaños de las dimensiones, este resultado se ve más claro.

## Desarrollo del algoritmo

### Versión para un máximo de 3 dimensiones

In [390]:
def t_data2(data, sub_shp, inverse=False):
    
    dim = [0, 0, 0, 0]
    sho = [1, 1, 1, 1]
    shp = [1, 1, 1, 1]
    sub = [1, 1, 1, 1]
    
    dest = np.zeros(data.shape, dtype=data.dtype).flatten()
       
    org_shp = list(data.shape)
        
    dta_shp = [org_shp[i] - org_shp[i]%sub_shp[i] for i in range(len(org_shp))]
    
    org_shp.reverse()   
    dta_shp.reverse()
    sub_shp = sub_shp
    sub_shp.reverse()
    
    for i in range(len(dta_shp)):
        dim[i] = 1
        shp[i] = dta_shp[i]
        sub[i] = sub_shp[i]
        sho[i] = org_shp[i]

    cont = 0
    
    K = 0    
    
    data_aux = data.flatten()
    
    while K <  shp[0]*shp[1]*shp[2]*shp[3]:
        
        if (((K + cont)%sho[0] >= sho[0] - sho[0]%sub[0])
            or ((K + cont)%(sho[0]*sho[1])//(sho[0]) >= sho[1] - sho[1]%sub[1])
            or ((K + cont)%(sho[0]*sho[1]*sho[2])//(sho[0]*sho[1]) >= sho[2] - sho[2]%sub[2])
            or ((K + cont)%(sho[0]*sho[1]*sho[2]*sho[3])//(sho[0]*sho[1]*sho[2]) >= sho[3] - sho[3]%sub[3])):                        
                       

            dest[K + cont] = data_aux[K + cont]
            
            cont += 1
        
        else:
            L = K + cont
  
            J = (dim[0]*((K)%sub[0] + K//(sub[0]*sub[1]*sub[2]*sub[3])%(shp[0]//sub[0])*sub[0])
                 +
                 dim[1]*(K//(sub[0])%sub[1]*shp[0] + K//(shp[0]*sub[1]*sub[2]*sub[3])%(shp[1]//sub[1])*shp[0]*sub[1])
                 +
                 dim[2]*(K//(sub[0]*sub[1])%sub[2]*shp[1]*shp[0] + K//(shp[0]*shp[1]*sub[2]*sub[3])%(shp[2]//sub[2])*shp[0]*shp[1]*sub[2])
                 +
                 dim[3]*(K//(sub[0]*sub[1]*sub[2])%sub[3]*shp[2]*shp[1]*shp[0] + K//(shp[0]*shp[1]*shp[2]*sub[3])%(shp[3]//sub[3])*shp[0]*shp[1]*sub[2]*sub[3])
                )
        
            inc = (J//shp[0] * (sho[0]%sub[0])
                   + J//(shp[1]*shp[0]) * (sho[1]%sub[1])
                   + J//(shp[2]*shp[1]*shp[0]) * (sho[2]%sub[2])
                   + J//(shp[3]*shp[2]*shp[1]*shp[0]) * (sho[3]%sub[3]))


            dest[L] = data_aux[J + inc]
                        
            K+=1
    
    # Añadimos los últimos elementos que faltan por copiar
    
    # OJO. Esto hay que corregirlo.
    
    L = K + cont
    for i in range(sho[0]%sub[0]):
        dest[L + i] = data_aux[L + i]
    return dest

In [389]:
# Definición de los datos

a, b, c, d = 4, 4, 4, 3

data = np.arange(a * b * c * d, dtype=np.int32).reshape(a, b, c, d)
sub_shape = [2, 2, 3, 2]

# Test de las dos funciones creadas

res = t_data2(data, sub_shape, inverse=False)

print(res.reshape(data.shape))

[[[[  0   1   2]
   [  3   4   5]
   [  6   7   8]
   [  9  10  11]]

  [[ 10  11  14]
   [ 13  14  17]
   [ 16  17  20]
   [ 21  22  23]]

  [[ 40  41  26]
   [ 43  44  29]
   [ 46  47  32]
   [ 33  34  35]]

  [[ 50  51  38]
   [ 53  54  41]
   [ 56  57  44]
   [ 45  46  47]]]


 [[[ 20  21  50]
   [ 23  24  53]
   [ 26  27  56]
   [ 57  58  59]]

  [[ 30  31  62]
   [ 33  34  65]
   [ 36  37  68]
   [ 69  70  71]]

  [[ 60  61  74]
   [ 63  64  77]
   [ 66  67  80]
   [ 81  82  83]]

  [[ 70  71  86]
   [ 73  74  89]
   [ 76  77  92]
   [ 93  94  95]]]


 [[[ 40  41  98]
   [ 43  44 101]
   [ 46  47 104]
   [105 106 107]]

  [[ 50  51 110]
   [ 53  54 113]
   [ 56  57 116]
   [117 118 119]]

  [[ 80  81 122]
   [ 83  84 125]
   [ 86  87 128]
   [129 130 131]]

  [[ 90  91 134]
   [ 93  94 137]
   [ 96  97 140]
   [141 142 143]]]


 [[[ 60  61 146]
   [ 63  64 149]
   [ 66  67 152]
   [153 154 155]]

  [[ 70  71 158]
   [ 73  74 161]
   [ 76  77 164]
   [165 166 167]]

  [[100 101 17